Agents are connected but have their own states. Only the final responses are appended to global state

<img src="../../resources/images/multi_agent_supervisor.png" alt="Multi Agent Supervisor" height="300" width="300">

Agents are independent LangChain agents. This means they have their own individual prompt, LLM, and tools.

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
from src.langchain.lg_workflow import *
from src.langchain.agents.framenet_agent import *
from src.langchain.agents.flanagan_agent import *
config = {"configurable" : {"thread_id" : 1}}

In [ ]:
fnr = framenet_tool.invoke({'instruction':'pick up the cup'})
fnr

In [2]:
# Example: Complex Query Using Multiple Agents
input_question = "what is 2 plus 2"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True,
    config=config,
):
    print(s)
    print("----")


Next Worker: mather
((), {'supervisor': None})
----
INSIDE ADD TOOL
(('mather:602afda2-fe2b-f36a-9631-e4cbd6d7ea1e',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WO6e3rVPumWplTjFxlA1PeFL', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 94, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6fc597af-f7c2-449c-82ad-30953974d890-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'call_WO6e3rVPumWplTjFxlA1PeFL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 18, 't

In [ ]:
print(graph.get_state(config))

In [2]:
# Example: Complex Query Using Multiple Agents
input_question = "framenet representation of the action instruction pour water from the bottle into the container."
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True,
    config=config,
):
    print(s)
    print("----")


Next Worker: framenet
((), {'supervisor': None})
----
INSIDE FRAMENET TOOL
(('framenet:1e22cc7b-4ea7-fec2-fcfa-9818bb786999',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:4b', 'created_at': '2025-05-16T09:25:46.759065362Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1387892955, 'load_duration': 8210143, 'prompt_eval_count': 189, 'prompt_eval_duration': 28310992, 'eval_count': 145, 'eval_duration': 1347833299, 'model_name': 'qwen3:4b'}, id='run--cbddf267-f2eb-434a-9d8d-cc0093fba1df-0', tool_calls=[{'name': 'framenet_tool', 'args': {'__arg1': 'pour water from the bottle into the container.'}, 'id': '6cb3e308-c9f9-4d8c-89e3-d1e984f8ba8d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 189, 'output_tokens': 145, 'total_tokens': 334})]}})
----
(('framenet:1e22cc7b-4ea7-fec2-fcfa-9818bb786999',), {'tools': {'messages': [ToolMessage(content='"{\\n  \\"framenet\\": \\"pouring\\",\\n  \\"frame\\": \\"Pouring\\",\\n 

In [3]:
graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='framenet representation of the action instruction pour water from the bottle into the container.', additional_kwargs={}, response_metadata={}, id='49ecf4fa-5ce7-4ced-81a4-319b1b1fbcfe'), HumanMessage(content='<think>\nOkay, let\'s see. The user asked for the FrameNet representation of the action instruction "pour water from the bottle into the container." I called the framenet_tool with that sentence, and the response came back with a structured JSON.\n\nFirst, I need to parse the tool\'s response. The main frame is "Pouring" with the lexical unit "pour.v". The core elements include the agent as "robot", theme_patient as "water", instrument as "bottle", source as "bottle", goal as "container", and result as "container has water". Then there\'s the peripheral information with location, manner, direction, time, quantity, and portion.\n\nNow, I need to present this information in a clear and concise way. The user probably wants to u

In [ ]:
graph.invoke({'messages': [HumanMessage(content='framenet representation of the action pick up the cup from the fridge')]}, config=config)['messages'][0].content

In [ ]:
graph.get_state(config=config)

In [ ]:
# graph.get_state(config=config).values['messages'][-1].content
framenet_answers

In [ ]:
graph.invoke({'messages': [HumanMessage(content='framenet representation of the action pick up the mug from the fridge')]})['messages'][1].content

In [2]:
# Example: Complex Query Using Multiple Agents
input_question = "flanagan representation of the action instruction pour water from the bottle into the container."
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True,
    config=config,
):
    print(s)
    print("----")


Next Worker: flanagan
((), {'supervisor': None})
----
INSIDE flanagan TOOL
(('flanagan:d861f995-50bb-e6cd-8a31-944f3293f18a',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:4b', 'created_at': '2025-05-16T09:31:33.464844328Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6640294565, 'load_duration': 8372327, 'prompt_eval_count': 186, 'prompt_eval_duration': 22334224, 'eval_count': 762, 'eval_duration': 6605886177, 'model_name': 'qwen3:4b'}, id='run--63584edf-9e8b-4591-984f-8ed1447b0ed4-0', tool_calls=[{'name': 'flanagan_tool', 'args': {'__arg1': 'pour water from the bottle into the container'}, 'id': 'e676eaa1-7dee-45e5-9e2d-7de3a2257d0f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 186, 'output_tokens': 762, 'total_tokens': 948})]}})
----
(('flanagan:d861f995-50bb-e6cd-8a31-944f3293f18a',), {'tools': {'messages': [ToolMessage(content='{"llm_flanagan": "task=\'pour water from the bottle into the container\' p

KeyboardInterrupt: 

In [ ]:
graph.invoke({'messages': [HumanMessage(content='flanagan representation of instruction pickup the bottle from the fridge')]}, config=config)['messages'][1].content

In [ ]:
flanagan_agent.invoke({'messages': [HumanMessage(content='flanagan representation of instruction pour water from the bottle into the container')]})

In [ ]:
from IPython.display import display, Image
# display(Image(math_agent.get_graph().draw_mermaid_png()))
# display(Image(websearch_agent.get_graph().draw_mermaid_png()))
# display(Image(framenet_agent.get_graph().draw_mermaid_png()))
# display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Example: Complex Query Using Multiple Agents
input_question = "what is 2 times 2"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True
):
    print(s)
    print("----")


In [ ]:
# Example: Complex Query Using Multiple Agents
input_question = "what is the value of 2 + 10"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True
):
    print(s)
    print("----")


In [ ]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is apple company founder"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")


In [ ]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is the ceo of google and what is the framenet representation of apple"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")


In [ ]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is the ceo of google, what is the framenet representation of apple and how much is 2 times 10"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")
